In [ ]:
from utils import *
import torch

In [ ]:
# Load EEG data
path = './data/'
winLength = 2
numChan = 24
srate = 128
feature = 'topo'
one_channel = True

In [ ]:
role = 'train'
train_data = load_data(path, role, winLength, numChan, srate, feature, one_channel)
print(f'X_train shape: {len(train_data)}, {train_data[0][0].shape}')
print(f'Y_train shape: {len(train_data)}, {train_data[0][1].shape}')

role = 'val'
val_data = load_data(path, role, winLength, numChan, srate, feature, one_channel)
print(f'X_val shape: {len(val_data)}, {val_data[0][0].shape}')
print(f'Y_val shape: {len(val_data)}, {val_data[0][1].shape}')
plot_EEG(train_data, feature, numChan, one_channel)

In [ ]:
x_data = train_data[:][0]
sample = 2
image = np.reshape(x_data[sample], (x_data[sample].shape[1],x_data[sample].shape[2]))
plt.imshow(image.astype('int32'), cmap='gray', vmin=0, vmax=255)

In [ ]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

In [ ]:
model = create_model('original',feature)
from pytorch_model_summary import summary
print(summary(model, torch.zeros((1, 1, 24, 72)), show_input=False))

In [ ]:
logger = Logger()

batch_size = 70 # original
loader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(val_data, batch_size=batch_size)
for s in range(10):
    model = run_experiment(s, 'original', 'topo',70)

In [ ]:
logger = Logger(mode='debug')
role = 'test'
test_data = load_data(path, role, winLength, numChan, srate, feature, one_channel)

sample_acc, subject_acc = test_all_seeds("saved-model/original-topo/model-original-topo", 'original',feature,test_data,'data/test_subjIDs_fewer_subjects.csv',10, 10,device, dtype,logger)
min_sample = np.min(sample_acc)
max_sample = np.max(sample_acc)
mean_sample = np.mean(sample_acc)
std_sample = np.std(sample_acc)

min_subj = np.min(subject_acc)
max_subj = np.max(subject_acc)
mean_subj = np.mean(subject_acc)
std_subj = np.std(subject_acc)

logger.log("Per sample")
logger.log(f"Min: {min_sample}, Max: {max_sample}, Mean: {mean_sample}, STDEV: {std_sample}")

logger.log("Per subject")
logger.log(f"Min: {min_subj}, Max: {max_subj}, Mean: {mean_subj}, STDEV: {std_subj}")


In [ ]:
logger = Logger(mode='debug')
epochs = [20, 30, 40, 60, 69]

with open("original-topo-test-results-balanced.csv", 'w') as out:
    out.write('epoch,min_sam,max_sam,mean_sam,std_sam,min_subj,max_subj,mean_subj,std_subj\n')
    for epoch in epochs:
        sample_acc, subject_acc = test_all_seeds(model_name="original-topo/model-original-topo", epoch=epoch, isBalanced=True)

        min_sample = np.min(sample_acc)
        max_sample = np.max(sample_acc)
        mean_sample = np.mean(sample_acc)
        std_sample = np.std(sample_acc)

        min_subj = np.min(subject_acc)
        max_subj = np.max(subject_acc)
        mean_subj = np.mean(subject_acc)
        std_subj = np.std(subject_acc)

        logger.log("Per sample")
        logger.log(f"Min: {min_sample}, Max: {max_sample}, Mean: {mean_sample}, STDEV: {std_sample}")

        logger.log("Per subject")
        logger.log(f"Min: {min_subj}, Max: {max_subj}, Mean: {mean_subj}, STDEV: {std_subj}")
        out.write(f"{epoch},{min_sample},{max_sample},{mean_sample},{std_sample},{min_subj},{max_subj},{mean_subj},{std_subj}\n")
        